In [1]:
import sys,os,time,cv2
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

import ops
from resnet_tsm import resnet18 as resnet
from utils import get_dtstr, emo2txt, imread_to_rgb, crop_img
import face_recognition
import moviepy.editor as mpe

# restore net
net = resnet().cuda()
ckpt = torch.load(ops.weight_path+'/resnet18_tsm_weights.tar')
net.load_state_dict(ckpt['model_state_dict'])
net.eval()
print 'net loaded'

net loaded


In [2]:
# episode_path
# DB_PATH = '/home/jhchoi/datasets2/friends/'
epi_sel = 'ep06'
DB_PATH = '../../img_ep06/'

# output path
OUT_PATH = '../data/friends_s01_'+epi_sel+'.jsonl'
out_json = open(OUT_PATH, 'w+')

import json

ep06_track = sorted(os.listdir('../../anno_ep06/'))
ep06_dict = dict()
for i,f in enumerate(ep06_track):
    with open('../../anno_ep06/'+f) as jsf:
        js = json.load(jsf)
        ep06_dict[i] = js


In [ ]:
for i,imf in enumerate(sorted(os.listdir(DB_PATH))):
    sys.stdout.write("\r"+str(i)+'/'+str(len(os.listdir(DB_PATH))))
    f_b_size = 4      # buffer size
    
    id_dict = ep06_dict[i]
    
    # new frame buffer
    f_buffer = []
    for j in range(f_b_size):
        f_fname = imf #str('%05d'%(i-f_b_size+j+1))+'.jpg'
        f_img = imread_to_rgb(os.path.join(DB_PATH, f_fname))
        f_buffer.append(f_img)
    
    # === process buffer
    for j in range(len(id_dict)):
        # crop person region coor=(xmin,ymin,xmax,ymax)
#         obj_coor = np.array([id_dict[j]['topleft']['x'],id_dict[j]['topleft']['y'], 
#                              id_dict[j]['bottomright']['x'],id_dict[j]['bottomright']['y'] ]).astype(int)
        obj_coor = np.array([id_dict[j]['bottomright']['x'],id_dict[j]['bottomright']['y'], 
                             id_dict[j]['topleft']['x'],id_dict[j]['topleft']['y'] ]).astype(int)
    
        obj_crop = crop_img(f_buffer[-1], obj_coor[0], obj_coor[1], obj_coor[2]-obj_coor[0], obj_coor[3]-obj_coor[1])
        obj_id = int(id_dict[j]['id'])
        # detect faces
        faces_coor = np.array(face_recognition.face_locations((obj_crop*255).astype(np.uint8))).astype(int)
        if faces_coor.ndim < 2:
            # no face, continue
            continue
        else:
            num_faces = faces_coor.shape[0]
        # refine coordinates as (xmin,ymin,xmax,ymax)
        faces_coor = faces_coor[0]
        faces_coor = np.array([faces_coor[3], faces_coor[0], faces_coor[1], faces_coor[2]])
        faces_coor[0] += obj_coor[0]
        faces_coor[1] += obj_coor[1]
        faces_coor[2] += obj_coor[0]
        faces_coor[3] += obj_coor[1]

        # extract faces and get emotions
        # get face crops from buffer
        coor = faces_coor #faces_coor[:,j]
        w = coor[2] - coor[0]
        h = coor[3] - coor[1]
        m = 0.5
        s = np.sqrt((w+(w+h)*m)*(h+(w+h)*m))

        f_crops = [crop_img(im, int(coor[0]+w*0.5), int(coor[1]+h*0.5), int(s), int(s), True) for im in f_buffer]
        f_batch = [cv2.resize(f_c, (224,224)) for f_c in f_crops]
        #plt.imsave('../../img_crop_ep06/'+str('%05d'%i)+'_'+str(j)+'.jpg', f_batch[0])
        # net forward - get emotion
        f_batch = torch.Tensor(np.array(f_batch)).cuda()
        f_batch = f_batch.unsqueeze(0).permute(0,1,4,2,3)
        with torch.no_grad():
            f_emo = net(f_batch)[0,-2:,:].mean(0).argmax().detach().cpu().numpy()
        # write text
        f_emo_txt =  emo2txt(f_emo)
        json_txt = str('{"type": "emotion", "class": "%s", "frames": %d, "coordinates": [%d,%d,%d,%d], "id": %d}\n'%\
                       (f_emo_txt, i, coor[0],coor[1],coor[2],coor[3], obj_id))
        out_json.write(json_txt)

out_json.close()
#float(i-1)*29907/6237.*1/23.98

6038/6237